<a href="https://colab.research.google.com/github/efcor/tf-sandbox/blob/main/model-attempt-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google colab includes tensorflow 2.17 but that has a bug with FactorizedTopK,
# so force the runtime to downgrade tensorflow via uninstall and reinstall.
# Note: requires a restart of the runtime.
!pip uninstall tensorflow_recommenders
!pip uninstall tensorflow
!pip install tensorflow==2.12
!pip install tensorflow_recommenders

In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Recommenders version:", tfrs.__version__)
# Good to go if this outputs:
# TensorFlow version: 2.12.0
# Recommenders version: v0.7.3

In [ ]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [5]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [6]:
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [7]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [8]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [9]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [10]:
print(unique_movie_titles[:10])

[b"'Til There Was You (1997)" b'1-900 (1994)' b'101 Dalmatians (1996)'
 b'12 Angry Men (1957)' b'187 (1997)' b'2 Days in the Valley (1996)'
 b'20,000 Leagues Under the Sea (1954)' b'2001: A Space Odyssey (1968)'
 b'3 Ninjas: High Noon At Mega Mountain (1998)' b'39 Steps, The (1935)']


In [11]:
embedding_dimension = 32

In [12]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [15]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [16]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [17]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [18]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=3)

In [20]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 9s 1s/step - factorized_top_k/top_1_categorical_accuracy: 9.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0093 - factorized_top_k/top_10_categorical_accuracy: 0.0223 - factorized_top_k/top_50_categorical_accuracy: 0.1250 - factorized_top_k/top_100_categorical_accuracy: 0.2353 - loss: 31088.4310 - regularization_loss: 0.0000e+00 - total_loss: 31088.4310


{'factorized_top_k/top_1_categorical_accuracy': 0.0009500000160187483,
 'factorized_top_k/top_5_categorical_accuracy': 0.00930000003427267,
 'factorized_top_k/top_10_categorical_accuracy': 0.02225000038743019,
 'factorized_top_k/top_50_categorical_accuracy': 0.125,
 'factorized_top_k/top_100_categorical_accuracy': 0.2353000044822693,
 'loss': 28245.9453125,
 'regularization_loss': 0,
 'total_loss': 28245.9453125}

In [21]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Rudy (1993)' b"Kid in King Arthur's Court, A (1995)"
 b'Aristocats, The (1970)']


In [34]:
# Export the query model. In Colab, this saves
# to /content/movie_recommender_model
# tf.saved_model.save(index, 'movie_recommender_model')

# Load it back; can also be done in TensorFlow Serving.
loaded = tf.saved_model.load('movie_recommender_model')

# Pass a user id in, get top predicted movie titles back.
user_id = ["42"]
scores, titles = loaded(user_id)

print(f"Recommendations: {titles[0][:10]}")

Recommendations: [b'Rudy (1993)' b"Kid in King Arthur's Court, A (1995)"
 b'Aristocats, The (1970)' b'With Honors (1994)' b'Maverick (1994)'
 b'Father of the Bride Part II (1995)' b'Rent-a-Kid (1995)'
 b'Jumanji (1995)' b'Jungle Book, The (1994)' b'Sabrina (1995)']
